# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
cities_df = pd.read_csv("../WeatherPY/output_data/cities.csv")
cities_df.drop(cities_df.columns[[0]], axis=1, inplace=True)
cities_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df['Humidity']
fig = gmaps.figure(zoom_level = 2, center = (46.0, -5.0))
heat_layer = gmaps.heatmap_layer(locations, weights = humidity , dissipating = False, max_intensity = 10, point_radius =1)
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
cleaned_cities_df = cities_df.loc[ (cities_df["Cloudiness"] < 10) & (cities_df["Max Temp"] < 302) & (cities_df["Max Temp"] > 299), :]
cleaned_cities_df.reset_index(drop=True, inplace=True)
cleaned_cities_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = cleaned_cities_df
hotel_df["Hotel Name"] = ''


In [ ]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
    }

for index, row in hotel_df.iterrows():
    
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    params["location"] = f"{Lat},{Lng}",
    
    print(f'Retrieving results for Index:{index}:{Lat},{Lng}.')
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"First Hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except:
        print("Missing Hotel ... skipping.")
        
    print("--------------")    



    

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
